In [ ]:
import sys
sys.path.append('../build')
import IPSModule as ips
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import lattpy as lp

In [ ]:
gamma = 1
temperature = 1
rad = 10.0

epsilon = 1.0
sigma = 1.0

init_dis = 2 ** (1.0 / 6.0) * sigma
np.random.seed(42)  # for reproducibility
print(init_dis)

# generate the initial configuration
_dis = init_dis * 2 / np.sqrt(3)
# _dis = init_dis * 2

latt = lp.Lattice.hexagonal(a = _dis)
# latt = lp.Lattice.square(a = _dis)

latt.add_atom()
latt.add_connections()
# s = lp.Circle((1, 1), radius=6)
s = lp.Donut((0, 0), radius_outer = 6, radius_inner = 2)

latt.build(shape = s, primitive=True)
ax = latt.plot()
s.plot(ax)
plt.show()

num_particles = latt.data.positions.shape[0]

init_particles_positions = latt.data.positions
p = ips.LangevinSystem(num_particles, gamma, temperature)

for i in range(num_particles):
    for d in range(2):
        p.get_positions()[d][i] = init_particles_positions[i][d] / 2
        p.get_velocities()[d][i] = 0.0

pair_force_config = {
    "type": "LennardJones",
    "eps": epsilon,
    "sigma": sigma
}

confinement_config = {
    "type": "Radial",
    "rad": rad
}
simulator = ips.IPS_Simulator_Langevin(p)
simulator.init(pair_force_config, confinement_config)

In [ ]:
from utils import SimulationVisualizer

num_frames = 100
num_step = 150000
sim_visualizer = SimulationVisualizer(simulator=simulator, particle_system=p, rad = 10, draw_interval=num_step // num_frames, dt = 0.001)

html = sim_visualizer.run_animation(num_step)
display(html)
# sim_visualizer.run_animation_to_gif(num_step, "test1.gif")

The code for two same clusters:

In [ ]:
gamma = 1       # Increase damping to reduce energy dissipation
temperature = 1
rad = 10.0

epsilon = 1.0   # Decrease the attractive strength of the Lennard-Jones potential
sigma = 1.0     # Increase the interaction distance between particles

init_dis = 2 ** (1.0 / 6.0) * sigma
np.random.seed(42)  # for reproducibility
print(init_dis)

# Generate the initial configuration
_dis = init_dis * 2 / np.sqrt(3)

latt = lp.Lattice.hexagonal(a=_dis)
latt.add_atom()
latt.add_connections()
s = lp.Donut((0, 0), radius_outer=6, radius_inner=2)

latt.build(shape=s, primitive=True)
ax = latt.plot()
s.plot(ax)
plt.show()

num_particles = latt.data.positions.shape[0]
init_particles_positions = latt.data.positions

# Define initial velocity and cluster positions
v0 = 0.5  # Velocity magnitude
dx = 3.0  # Distance between the two clusters

# Copy the initial particle positions and generate the second cluster
positions1 = init_particles_positions  # First cluster positions
positions2 = init_particles_positions + np.array([dx, 0])  # Second cluster, shifted to the right by dx

# Merge the particle positions of both clusters
all_positions = np.vstack([positions1, positions2])

# Update LangevinSystem
num_particles = all_positions.shape[0]
p = ips.LangevinSystem(num_particles, gamma, temperature)

# Set positions and velocities (no division by 2 to ensure correct initialization)
for i in range(num_particles):
    for d in range(2):
        p.get_positions()[d][i] = all_positions[i][d]  # Correction: No position scaling
    
    # Assign velocities
    if i < len(positions1):  # First cluster
        p.get_velocities()[0][i] = v0  # Positive velocity in the x direction
        p.get_velocities()[1][i] = 0.0  # Keep velocity in the y direction at 0
    else:  # Second cluster
        p.get_velocities()[0][i] = -v0  # Negative velocity in the x direction
        p.get_velocities()[1][i] = 0.0  # Keep velocity in the y direction at 0

# Configure the force field
pair_force_config = {
    "type": "LennardJones",
    "eps": epsilon,
    "sigma": sigma
}

confinement_config = {
    "type": "Radial",
    "rad": rad
}

simulator = ips.IPS_Simulator_Langevin(p)
simulator.init(pair_force_config, confinement_config)

# Adjust animation settings
from utils import SimulationVisualizer

num_frames = 100
num_step = 150000

# Reinitialize the visualizer to ensure all particles are included
sim_visualizer = SimulationVisualizer(
    simulator=simulator,
    particle_system=p,
    rad=10,
    draw_interval=num_step // num_frames,
    dt=0.0001  # Reduce time step to avoid numerical errors
)

# Ensure `p` still contains all particles after initializing SimulationVisualizer
html = sim_visualizer.run_animation(num_step)
display(html)
# sim_visualizer.run_animation_to_gif(num_step, "test1.gif")


For two different clusters:

In [ ]:
gamma = 1
temperature = 1
rad = 25.0

epsilon = 1.0  # Decrease the attractive strength of the Lennard-Jones potential
sigma = 1.0  # Increase the interaction distance between particles

dt = 0.0005  # Reduce the time step to avoid numerical errors
gamma = 1.0  # Increase damping to reduce energy dissipation

init_dis = 2 ** (1.0 / 6.0) * sigma
np.random.seed(42)  # for reproducibility
print(init_dis)

# Generate the initial configuration
_dis = init_dis * 2 / np.sqrt(3)

# Generate the first cluster
latt1 = lp.Lattice.hexagonal(a=_dis)
latt1.add_atom()
latt1.add_connections()
s1 = lp.Donut((0, 0), radius_outer=8, radius_inner=3)
latt1.build(shape=s1, primitive=True)
ax1 = latt1.plot()
s1.plot(ax1)
plt.show()

positions1 = latt1.data.positions
num_particles1 = positions1.shape[0]

# Generate the second, smaller cluster
latt2 = lp.Lattice.hexagonal(a=_dis)
latt2.add_atom()
latt2.add_connections()
s2 = lp.Donut((0, 0), radius_outer=4, radius_inner=2)  # Set a smaller size
latt2.build(shape=s2, primitive=True)
ax2 = latt2.plot()
s2.plot(ax2)
plt.show()

positions2 = latt2.data.positions + np.array([15.0, 0])  # Shift to the right to form the second cluster, adjust position here
num_particles2 = positions2.shape[0]

# Merge the particle positions of both clusters
all_positions = np.vstack([positions1, positions2])

# Update LangevinSystem
num_particles = all_positions.shape[0]
p = ips.LangevinSystem(num_particles, gamma, temperature)

# Set positions and velocities (no division by 2 to ensure correct initialization)
for i in range(num_particles):
    for d in range(2):
        p.get_positions()[d][i] = all_positions[i][d]  # Correction: No position scaling
    
    # Assign velocities
    if i < num_particles1:  # First cluster
        p.get_velocities()[0][i] = 2.0  # Positive velocity in the x direction
        p.get_velocities()[1][i] = 0.0  # Keep velocity in the y direction at 0
    else:  # Second, smaller cluster
        p.get_velocities()[0][i] = -2.0  # Negative velocity in the x direction
        p.get_velocities()[1][i] = 0.0  # Keep velocity in the y direction at 0

# Configure the force field
pair_force_config = {
    "type": "LennardJones",
    "eps": epsilon,
    "sigma": sigma
}

confinement_config = {
    "type": "Radial",
    "rad": rad
}

simulator = ips.IPS_Simulator_Langevin(p)
simulator.init(pair_force_config, confinement_config)

# **Fix animation settings**
from utils import SimulationVisualizer

num_frames = 100
num_step = 15000000

# Reinitialize the visualizer to ensure all particles are included
sim_visualizer = SimulationVisualizer(
    simulator=simulator,
    particle_system=p,
    rad=25,  # Adjust the radius here as well
    draw_interval=num_step // num_frames,
    dt=dt
)

# Ensure `p` still contains all particles after initializing SimulationVisualizer
html = sim_visualizer.run_animation(num_step)
display(html)
# sim_visualizer.run_animation_to_gif(num_step, "test1.gif")
